In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
neutral_model = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb")
# positive_model = AutoModelForCausalLM.from_pretrained("/home/advaitmb/notebooks/projects/PreDiction/nbs/gpt2-imdb-positive-sentiment")
# negative_model = AutoModelForCausalLM.from_pretrained("/home/advaitmb/notebooks/projects/PreDiction/nbs/gpt2-imdb-negative-sentiment")
neutral_model.to('cuda')
# positive_model.to('cuda')
# negative_model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [2]:
device = 'cuda'

In [13]:
# from datasets import load_dataset
# test = load_dataset('imdb', split='test')
# encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')

Reusing dataset imdb (/home/advaitmb/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


In [4]:
from fastai.text.all import *
from tqdm import tqdm

In [5]:
positive_test_path = '/home/advaitmb/.fastai/data/imdb/test/pos/'
positive_test_files = get_text_files(positive_test_path)

negative_test_path = '/home/advaitmb/.fastai/data/imdb/test/neg/'
negative_test_files = get_text_files(negative_test_path)

In [6]:
def files_to_list(files):
    ls = []
    for file in files:
        with open(file, 'r') as f:
            ls.append(f.read())
    return ls


In [8]:
positive_test_text_list = files_to_list(positive_test_files)
negative_test_text_list = files_to_list(negative_test_files)

In [ ]:
positive_model_test_encodings = tokenizer('\n\n'.join(positive_test_text_list), return_tensors='pt')

In [21]:
negative_model_test_encodings = tokenizer('\n\n'.join(negative_test_text_list), return_tensors='pt')

Token indices sequence length is longer than the specified maximum sequence length for this model (3680989 > 1024). Running this sequence through the model will result in indexing errors


In [19]:
def compute_perplexity(model, encodings):
    max_length = model.config.n_positions
    stride = 512

    lls = []
    for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i    # may be different from stride on last loop
        input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs[0] * trg_len

        lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    return ppl

In [18]:
positive_model_perplexity = compute_perplexity(positive_model, positive_model_test_encodings)

100%|██████████| 7207/7207 [28:31<00:00,  4.21it/s]


In [19]:
positive_model_perplexity

tensor(36.9713, device='cuda:0')

In [22]:
negative_model_perplexity = compute_perplexity(negative_model, negative_model_test_encodings)

100%|██████████| 7190/7190 [28:28<00:00,  4.21it/s]


In [23]:
negative_model_perplexity

tensor(34.6978, device='cuda:0')

In [13]:
neutral_test_text_list = positive_test_text_list[:int(len(positive_test_text_list)/2)]

In [15]:
neutral_test_text_list.extend(negative_test_text_list[:int(len(negative_test_text_list)/2)])

In [16]:
len(neutral_test_text_list)

12500

In [17]:
neutral_model_test_encodings = tokenizer('\n\n'.join(neutral_test_text_list), return_tensors='pt')

In [20]:
neutral_model_perplexity = compute_perplexity(neutral_model, neutral_model_test_encodings)

100%|██████████| 7166/7166 [28:43<00:00,  4.16it/s]


In [21]:
neutral_model_perplexity

tensor(32.2970, device='cuda:0')